In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
family_group = "9.Papilionidae"
annotated_genome_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/{family_group}/1.Blast_result"
species_list = os.listdir(annotated_genome_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
# species_list = ["Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Limenitis_camilla"]
# species_list = ["Argynnis_bischoffii_washingtonia","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Bicyclus_anynana","Coenonympha_glycerion","Elymnias_hypermnestra","Erebia_aethiops","Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Vanessa_tameamea"]
for species in species_list:
    print(species)
    blast_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/{family_group}/1.Blast_result"
    
    list_of_files = os.listdir(f"{blast_location}/{species}")
    query_species_list = []
    coordinate_detail_dictionary = {}
    coordinate_info_dictionary = {}
    scaffold = ''
    for file in list_of_files:
        if file.endswith(".csv") and len(file.split("_")) >= 5 and not(file.endswith("_old.csv")):
#             print(file)
            if "." in file[:-4]:
                query_species = f"{file.split('_')[-2].split('.')[1]}_{file.split('_')[-1].split('.')[0]}"
            else:
                query_species = f"{file.split('_')[-2]}_{file.split('_')[-1].split('.')[0]}"
            query_species_list.append(query_species)
            coordinate_detail_dictionary[query_species] = {}
            with open(f"{blast_location}/{species}/{file}", 'r') as coordinate_file_open:
                coordinate_file_lines_list  =  coordinate_file_open.readlines()
            if scaffold == '':
                scaffold = coordinate_file_lines_list[1].split(",")[1]
    
            for i in range(1,len(coordinate_file_lines_list)):
                
                coordinate_file_lines_split = coordinate_file_lines_list[i].split(",")
                start,stop = coordinate_file_lines_split[2],coordinate_file_lines_split[3]
                exon = f"Exon_{i}"
                if exon not in coordinate_info_dictionary:
                    coordinate_info_dictionary[exon] = {}
                coordinate_info_dictionary[exon][query_species] = [start,stop]
                coordinate_detail_dictionary[query_species][exon] = coordinate_file_lines_list[i]
            
            # break
            # print(query_species)
            # print(file)
    # print(coordinate_info_dictionary)
    # assert False
    output = coordinate_file_lines_list[0]
    for exon,species_specific_coordinates in coordinate_info_dictionary.items():
        # print(exon,species_specific_coordinates)
        coordinates_list = []
        previous_coordinates = ""
        match = 1
        for query_species, coordinates in coordinate_info_dictionary[exon].items():
            coordinates_list.append([query_species,coordinates])
            if previous_coordinates != '':
                if coordinates != previous_coordinates:
                    match = 0
                    
            previous_coordinates = coordinates
       
        
        
        if match == 0:
            print(exon,"mismatch")
            print(coordinates_list)
                
            while True:
                choice = input("Proceed?")
                if choice[0].lower() == "y" or choice[0].lower() == "n":
                    break
            
            while True:
                if choice[0].lower() == "y":
                    alignment_detail_dictionary = {}
                    for query_species_name in query_species_list:
                        list_of_folders  = os.listdir(f"{blast_location}/{species}")
                        for folders in list_of_folders:
                            if folders.endswith(query_species_name):
                                # print()
                                
                                with open(f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query.txt", 'r') as query_file_open:
                                    # alignment_detail = query_file_open.readlines()[0]
                                    alignment_detail = query_file_open.readlines()[0].split("set")[1][:-1]
                                    alignment_detail_dictionary[query_species_name] = alignment_detail
                                with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/{exon}_translated_genomic_sequence_{alignment_detail}.fa.hat2", 'r') as alignment_score_file:
                                    distance_score = float(alignment_score_file.readlines()[-1].rstrip())
                                try:
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query_spliced.txt", 'r') as new_query_file:
                                        new_query = (new_query_file.readlines()[-1].rstrip())
                                except:
                                    new_query = "Issue with query"
                                print(query_species_name,exon, alignment_detail, distance_score,f"\n{new_query}")
                    while True:            
                        check_alignment = input("Check Alignment??")
                        if check_alignment[0].lower() == "y" or check_alignment[0].lower() == "n":
                            break
                    if check_alignment[0].lower() == "y":
                        for query_species_name in query_species_list:
                            list_of_folders  = os.listdir(f"{blast_location}/{species}")
                            for folders in list_of_folders:
                                if folders.endswith(query_species_name):
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/alignment_clustal_{exon}_translated_genomic_sequence_{alignment_detail_dictionary[query_species_name]}.fa.txt", 'r') as clustal_alignment_file:
                                        clustal_alignment = clustal_alignment_file.readlines()
                                    print("".join(clustal_alignment))
                                    
                                # assert False
                    while True:
                        try:
                            species_number = (input(f"Choose Species 1 - {len(query_species_list)}"))
                            
                            
                            if species_number[0].lower() == 'n':
                                output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                                break 
                            output += coordinate_detail_dictionary[query_species_list[int(species_number)-1]][exon]
                            break
                        except:
                            print("Retry")
                    break
                elif choice[0].lower() == "n":
                    output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                    break
                
                
        else:
            print(exon, "Match")
            print(coordinates_list)
            output += coordinate_detail_dictionary[query_species_list[0]][exon]
        
       

        # assert False
    # print(coordinate_detail_dictionary)
    print(output)
    with open(f"{blast_location}/{species}/final_coordinates.csv", "w") as out_file:
        out_file.write(output)
    
    # 'Coenonympha_glycerion,OY979626.1,7015875,7015963,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29\n'

Battus_philenor
Exon_1 Match
[['Papilio_machaon', ['17013501', '17013589']], ['Papilio_polytes', ['17013501', '17013589']], ['Papilio_xuthus', ['17013501', '17013589']]]
Exon_2 Match
[['Papilio_machaon', ['17013868', '17013945']], ['Papilio_polytes', ['17013868', '17013945']], ['Papilio_xuthus', ['17013868', '17013945']]]
Exon_3 Match
[['Papilio_machaon', ['17016936', '17017099']], ['Papilio_polytes', ['17016936', '17017099']], ['Papilio_xuthus', ['17016936', '17017099']]]
Exon_4 mismatch
[['Papilio_machaon', ['17018286', '17018386']], ['Papilio_polytes', ['17018286', '17018392']], ['Papilio_xuthus', ['17018286', '17018386']]]
Proceed?y
Papilio_machaon Exon_4 83_frame2 1.108 
DLPSPALDSSEKGPESMDIATPDSGSTKEEIAI
Papilio_polytes Exon_4 83_frame2 1.09 
DLPSPALDSSEKGPESMDIATPDSGSTKEEIAIVS
Papilio_xuthus Exon_4 83_frame2 1.012 
DLPSPALDSSEKGPESMDIATPDSGSTKEEIAI
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_machaon VA----------------------LPSLPLASPDKGSESMDISTA

Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_machaon H--------------QGVKRASNQIETESVPNKLQCTAPRQLHRQQPTTTASSLTVTTPA
set68_frame0    GPFKYVKVFINIFQPHGVKRHSKHSDPTPGMHKHHCSSSRQTRRRHPTSSANPHPAPIPT
                               :**** *:: :. .  :* :*::.** :*::**::*.. ... *:

Papilio_machaon TTTA----------------------------------
set68_frame0    SSAGEDYLIFAVIITKGYCMASFSHHHLLNLKDTPNYY
                :::.                                  

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_polytes --------------PQGVKRSSKQIDLESVAHKHHCTTPRQTRRKQPITTM-PHTITTPA
set68_frame0    GPFKYVKVFINIFQPHGVKRHSKHSDPTPGMHKHHCSSSRQTRRRHPTSSANPHPAPIPT
                              *:**** **: *  .  *****::.*****::* ::  **. . *:

Papilio_polytes TTI-----------------------------------
set68_frame0    SSAGEDYLIFAVIITKGYCMASFSHHHLLNLKDTPNYY
                ::                                    

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_xuthus_ ----------

Exon_1 Match
[['Papilio_machaon', ['11537843', '11537931']], ['Papilio_polytes', ['11537843', '11537931']], ['Papilio_xuthus', ['11537843', '11537931']]]
Exon_2 Match
[['Papilio_machaon', ['11536672', '11536749']], ['Papilio_polytes', ['11536672', '11536749']], ['Papilio_xuthus', ['11536672', '11536749']]]
Exon_3 Match
[['Papilio_machaon', ['11536226', '11536380']], ['Papilio_polytes', ['11536226', '11536380']], ['Papilio_xuthus', ['11536226', '11536380']]]
Exon_4 Match
[['Papilio_machaon', ['11535381', '11535478']], ['Papilio_polytes', ['11535381', '11535478']], ['Papilio_xuthus', ['11535381', '11535478']]]
Exon_5 mismatch
[['Papilio_machaon', ['12271506', '12271589']], ['Papilio_polytes', ['11534732', '11534830']], ['Papilio_xuthus', ['12271506', '12271589']]]
Proceed?y
Papilio_machaon Exon_5 95_frame1 1.564 

Papilio_polytes Exon_5 66_frame2 1.645 
KIMTRGVTFQGALQFCRGDCHVFTFHSFSCRPT
Papilio_xuthus Exon_5 11_frame1 1.473 
QPVHVRLML
Check Alignment??y
CLUSTAL format alignment by MAFFT 

Choose Species 1 - 33
Exon_17 mismatch
[['Papilio_machaon', ['11522483', '11522681']], ['Papilio_polytes', ['11522492', '11522681']], ['Papilio_xuthus', ['11522492', '11522681']]]
Proceed?y
Papilio_machaon Exon_17 43_frame2 1.248 
THGVKRVSNQIESKEGAHKHHCASPRSPQHKQPTAITSAGKQSFYDYANFFLFIFLFMARSYYLHI
Papilio_polytes Exon_17 43_frame2 1.182 
THGVKRVSNQIESKEGAHKHHCASPRSPQHKQPTAITSAGKQSFYDYANFFLFIFLFMARSYY
Papilio_xuthus Exon_17 43_frame2 1.016 
THGVKRVSNQIESKEGAHKHHCASPRSPQHKQPTAITSAGKQSFYDYANFFLFIFLFMARSYY
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_machaon ----------------------------HQGVKRASNQIETESVPNKLQCTAPRQLHRQQ
set43_frame2    INTEFQLTIPVFLSWRDTTVYLRGTSLQTHGVKRVSNQIESKEGAHKHHCASPRSPQHKQ
                                             :****.*****::. .:* :*::**. :::*

Papilio_machaon PTTTASS--------------------------LTVTTPATTTA------
set43_frame2    PTAITSAGKQSFYDYANFFLFIFLFMARSYYLHIFISLYDTDNVGYFLNF
                **: :*:                          : ::  

Exon_1 Match
[['Papilio_machaon', ['254051', '254130']], ['Papilio_polytes', ['254051', '254130']], ['Papilio_xuthus', ['254051', '254130']]]
Exon_2 Match
[['Papilio_machaon', ['256470', '256544']], ['Papilio_polytes', ['256470', '256544']], ['Papilio_xuthus', ['256470', '256544']]]
Exon_3 mismatch
[['Papilio_machaon', ['257884', '258050']], ['Papilio_polytes', ['257863', '258050']], ['Papilio_xuthus', ['257863', '258050']]]
Proceed?y
Papilio_machaon Exon_3 76_frame0 1.049 
NNLSQPPEKRLKENDAKKKKQVQSENLTIEAKTEEEGRLPEPVQILESLKEDTSD
Papilio_polytes Exon_3 76_frame0 0.984 
HFFNSHSNNLSQPPEKRLKENDAKKKKQVQSENLTIEAKTEEEGRLPEPVQILESLKEDTSD
Papilio_xuthus Exon_3 76_frame0 1.034 
HFFNSHSNNLSQPPEKRLKENDAKKKKQVQSENLTIEAKTEEEGRLPEPVQILESLKEDTSD
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_machaon ------------------------------------------------------------
set76_frame0    TEISTYTNSLFANEQNYSFNKSKITSMFISYIFKTKRKKTLIIYHMIVSLYYSHFFNSHS
                                  

...

Choose Species 1 - 3n
Exon_23 mismatch
[['Papilio_machaon', ['1688065', '1688094']], ['Papilio_polytes', ['1689354', '1689506']], ['Papilio_xuthus', ['4781380', '4781481']]]
Proceed?y
Papilio_machaon Exon_23 78_frame2 1.647 
HREIRSNTKG
Papilio_polytes Exon_23 84_frame1 1.634 
TRENIEKVFISTLTRRESNPGPRDGDILKNRCLGHSSTEVVLQNFDNLREL
Papilio_xuthus Exon_23 8_frame1 1.686 

Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_machaon ------------VCVTSELIYKYQIQTKSMEEVLSGDKQRLKQLEQ--------------
set78_frame2    HMYIKVFFNRHNAVMEKQLVKIH----REIRSNTKGDREKYKQKKKEHLDRCNATFQPTS
                            . : .:*:  :    :.:..  .**::: ** ::              

Papilio_machaon -----------------------------------
set78_frame2    VGINVAEAHMIKAKVYQLLYIKVNSDLWHFKLVSF
                                                   

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_polytes ----------------------------------VCVTSEL------------IYKYQIQ
set84_frame1    KYYVYANINAKVRGMGIGNNEGTT

Proceed?y
Papilio_machaon Exon_5 211_frame2 1.404 
GPSRKEPTTSSQDG
Papilio_polytes Exon_5 107_frame2 1.542 
AGLSDH
Papilio_xuthus Exon_5 211_frame2 1.4 
GPSRKEPTTSSQDG
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_machaon FDPATKSPATKTSHNTTNRPPICIP------------------
set211_frame2   -GPSRKEPTTSSQDGIAATTPLILPIFTSGGDHLPPYQLLTIL
                 .*: *.*:*.:... :  .*: :*                  

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_polytes FDPDITKMKSPVQKSFDNAITRSSLSCR----------
set107_frame2   L--DFMLTSRPIAGLSDHCLIRSTLTISAYQPICVSDV
                :  *:   . *:    *:.: **:*:            

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_xuthus_ FDPATKSPVHKTLENTTNRTPLPFP------------------
set211_frame2   -GPSRKEPTTSSQDGIAATTPLILPIFTSGGDHLPPYQLLTIL
                 .*: *.*. .: :. :  *** :*                  

Choose Species 1 - 3n
Exon_6 Match
[['Papilio_machaon', ['731811', '731942']], ['Papilio_polytes', ['731811', '731942']], ['Pap

Exon_1 Match
[['Papilio_machaon', ['1726296', '1726384']], ['Papilio_polytes', ['1726296', '1726384']], ['Papilio_xuthus', ['1726296', '1726384']]]
Exon_2 Match
[['Papilio_machaon', ['1726064', '1726141']], ['Papilio_polytes', ['1726064', '1726141']], ['Papilio_xuthus', ['1726064', '1726141']]]
Exon_3 Match
[['Papilio_machaon', ['1724994', '1725154']], ['Papilio_polytes', ['1724994', '1725154']], ['Papilio_xuthus', ['1724994', '1725154']]]
Exon_4 mismatch
[['Papilio_machaon', ['1723547', '1723644']], ['Papilio_polytes', ['1723526', '1723644']], ['Papilio_xuthus', ['1723547', '1723644']]]
Proceed?y
Papilio_machaon Exon_4 54_frame0 1.133 
VQMPSQALASPEQAPECMDIVTTESSNMKEEL
Papilio_polytes Exon_4 54_frame0 1.16 
VQMPSQALASPEQAPECMDIVTTESSNMKEELVIVSSTS
Papilio_xuthus Exon_4 54_frame0 1.096 
VQMPSQALASPEQAPECMDIVTTESSNMKEEL
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_machaon ------------------------------------------------------------
set54_frame0    NLRMYV

Choose Species 1 - 33
Exon_18 Match
[['Papilio_machaon', ['1712971', '1713078']], ['Papilio_polytes', ['1712971', '1713078']], ['Papilio_xuthus', ['1712971', '1713078']]]
Exon_19 Match
[['Papilio_machaon', ['1712553', '1712666']], ['Papilio_polytes', ['1712553', '1712666']], ['Papilio_xuthus', ['1712553', '1712666']]]
Exon_20 Match
[['Papilio_machaon', ['1711833', '1712005']], ['Papilio_polytes', ['1711833', '1712005']], ['Papilio_xuthus', ['1711833', '1712005']]]
Exon_21 Match
[['Papilio_machaon', ['1708314', '1708505']], ['Papilio_polytes', ['1708314', '1708505']], ['Papilio_xuthus', ['1708314', '1708505']]]
Exon_22 Match
[['Papilio_machaon', ['1707891', '1707977']], ['Papilio_polytes', ['1707891', '1707977']], ['Papilio_xuthus', ['1707891', '1707977']]]
Exon_23 Match
[['Papilio_machaon', ['1707647', '1707748']], ['Papilio_polytes', ['1707647', '1707748']], ['Papilio_xuthus', ['1707647', '1707748']]]
Exon_24 Match
[['Papilio_machaon', ['1707368', '1707514']], ['Papilio_polytes', ['17

Papilio_polytes Exon_8 114_frame1 0.638 
VFVGRMPPTLINYISQCLHFLNRTAYRIPVQPLENSVSPMVCRIRRYKDLSSGFAIKNSVITYMPFLLKLSFKNVNGEEGIVTYLVIQAAPFSSAYK
Papilio_xuthus Exon_8 114_frame1 0.599 
VFVGRMPPTLINYISQCLHFLNRTAYRIPVQPLENSVSPMVCRIRRYKDLSSGFAIKNSVITYMPFLLKLSFKNVNGEEGIVTYLVIQAAPFSSAYK
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_machaon -----------LGR----------------------------DNLVSSMVCRIRRYRDLA
set114_frame1   DLPNGCVSVVFVGRMPPTLINYISQCLHFLNRTAYRIPVQPLENSVSPMVCRIRRYKDLS
                           :**                            :* **.********:**:

Papilio_machaon SGYGVKNNAVSFMPFLLKLAFKNVNNEQCNEIYLVIQASPFFSAFK--
set114_frame1   SGFAIKNSVITYMPFLLKLSFKNVNGEEGIVTYLVIQAAPFSSAYKSK
                **:.:**..:::*******:*****.*:    ******:** **:*  

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Papilio_polytes -------------------------------------PEK--ENLVSSMVCRIRRYRDLA
set114_frame1   DLPNGCVSVVFVGRMPPTLINYISQCLHFLNRTAYRIPVQPLENSVSPMVCRIRRYKDLS
                